In [1]:

import numpy as np
import pickle
import pandas as pd


In [2]:


def load_embeddings(pickle_file_path):
    with open(pickle_file_path, 'rb') as pickle_file:
        loaded_data_dict = pickle.load(pickle_file)

    # Convert lists to numpy arrays
    data = loaded_data_dict["embedding"]
    labels = np.array(loaded_data_dict["target"])
    return data, labels, loaded_data_dict

In [48]:
# train data
def concat_embeddings(
        embedding,
        semantic_att,
        mode
        ):
    path_s = semantic_att['paths']
    path_e = embedding['paths']

    X_s = semantic_att['embedding']
    X_e = embedding['embedding']

    num_att = X_s.shape[1] + X_e.shape[1]
    feature_embeddings = np.empty((0, num_att))


    if mode == 'train':
        for sem, emb, x_s, x_e in zip(path_s, path_e, X_s, X_e):
            if sem != 'train_data/'+emb:
                raise Exception(f'images are not the same: {sem} - train_data/{emb}')
            
            concatenated_array = np.concatenate((x_s, x_e))
            feature_embeddings = np.vstack((feature_embeddings, concatenated_array))
    else:
        for sem, emb, x_s, x_e in zip(path_s, path_e, X_s, X_e):
            if sem != emb:
                raise Exception(f'images are not the same: {sem} - train_data/{emb}')
            
            concatenated_array = np.concatenate((x_s, x_e))
            feature_embeddings = np.vstack((feature_embeddings, concatenated_array))

    model_name = semantic_att['model']+'_'+embedding['model']
    data_dict = {
        "model": model_name,
        "embedding":feature_embeddings,
        "target":semantic_att['target'],
        "paths": semantic_att['paths'],
        "classes":semantic_att['classes']
    }

    with open(f'./{model_name}_{mode}.pickle', 'wb') as pickle_file:
        pickle.dump(data_dict, pickle_file)

In [34]:
path_test = '../evaluation/efficientnetb0_4096_autoencoder_test.pickle'
path_train = '../evaluation/efficientnetb0_4096_autoencoder_train.pickle'
_, _, encod_result_test = load_embeddings(path_test)
_, _, encod_result_train = load_embeddings(path_train)

path_test = '../evaluation/efficientnetb0_4096_pretrained_test.pickle'
path_train = '../evaluation/efficientnetb0_4096_pretrained_train.pickle'
_, _, triplet_result_test = load_embeddings(path_test)
_, _, triplet_result_train = load_embeddings(path_train)

path_test = '../evaluation/efficientnet_SwaV_test.pickle'
path_train = '../evaluation/efficientnet_SwaV_train.pickle'
_, _, swav_result_test = load_embeddings(path_test)
_, _, swav_result_train = load_embeddings(path_train)

path_test = '../semantic/semantic_test.pickle'
path_train = '../semantic/semantic_train.pickle'
_, _, semant_result_test = load_embeddings(path_test)
_, _, semant_result_train = load_embeddings(path_train)

In [50]:
concat_embeddings(
        embedding=encod_result_train,
        semantic_att=semant_result_train,
        mode='train'
        )
concat_embeddings(
        embedding=encod_result_test,
        semantic_att=semant_result_test,
        mode='test'
        )

concat_embeddings(
        embedding=triplet_result_train,
        semantic_att=semant_result_train,
        mode='train'
        )
concat_embeddings(
        embedding=triplet_result_test,
        semantic_att=semant_result_test,
        mode='test'
        )

concat_embeddings(
        embedding=swav_result_train,
        semantic_att=semant_result_train,
        mode='train'
        )
concat_embeddings(
        embedding=swav_result_test,
        semantic_att=semant_result_test,
        mode='test'
        )